In [1]:
import gymnasium as gym
import cookiedisaster
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
print(torch.__version__)

SEED=2
np.random.seed(SEED)

env = gym.make('cookiedisaster/GridWorld-v0')

2.2.2


#### seed vs not seed, doens't matter, still get random each time??

In [2]:
env.reset(seed=SEED) 

c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:131: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\spaces\box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


({'agent': {'pos': 4, 'vel': 0.0},
  'cookie': {'pos': 0.10111663773625759, 'time': 5}},
 {'distance': 3.898883362263742, 'steps': 0})

#### get the observation:
- the agents pos and vel
- the cookies pos and time remaining

In [3]:
env.reset(seed=SEED)[0]

{'agent': {'pos': 4, 'vel': 0.0},
 'cookie': {'pos': 0.9194711828695544, 'time': 5}}

In [4]:
env.step(2)

c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:131: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


({'agent': {'pos': 4.2, 'vel': 1.0},
  'cookie': {'pos': 0.9194711828695544, 'time': 5}},
 0,
 False,
 False,
 {'distance': 3.280528817130446, 'steps': 1})

In [5]:
# env.render()
DUMMY_STATE=env.step(0)
MAX_TIME=5*25 # 5 cookies elapse time
PUNISHMENT=-1 # may have to be adjusted

# chat lets gooo

In [6]:
def normalize(value, min_value, max_value, scale_min=-1, scale_max=1):
    return ((value - min_value) / (max_value - min_value)) * (scale_max - scale_min) + scale_min

def preprocess_state(state):
    # Assuming state is a dictionary like:
    # {'robot': {'pos': x, 'vel': y}, 'cookie': {'pos': z, 'time': w}}
    robot_pos = normalize(state[0]['agent']['pos'], 0, 10)
    robot_vel = normalize(state[0]['agent']['vel'], -7, 7)
    cookie_pos = normalize(state[0]['cookie']['pos'], 0, 10)
    cookie_time = normalize(state[0]['cookie']['time'], 0, 5)
    distance = robot_pos - cookie_pos
    direction = 1 if distance > 0 else -1
    
    # Return the normalized state as a numpy array
    return np.array([robot_pos, robot_vel, cookie_pos, cookie_time,distance, direction])


# def preprocess_state(state):
#     # Assuming state is a dictionary with the structure you've shown
#     agent_pos = state[0]['agent']['pos']
#     agent_vel = state[0]['agent']['vel']
#     cookie_pos = state[0]['cookie']['pos']
#     cookie_time = state[0]['cookie']['time']
#     distance = state[-1]['distance']
#     steps = state[-1]['steps']
    
#     # Combine these into a flat array of features
#     processed_state = np.array([agent_pos, agent_vel, cookie_pos, cookie_time, distance, steps])

#     return processed_state

def choose_action(state, weights, epsilon, num_actions):
    if np.random.rand() < epsilon:
        return np.random.choice(num_actions)
    else:
        # print('state',state)
        # print('weights',weights)
        q_values = np.dot(state, weights)
        # print('q_values',q_values)
        return np.argmax(q_values)

def update_weights(state, action, reward, next_state, weights, learning_rate, discount_factor):
    next_action = choose_action(next_state, weights, 0, len(weights[0]))
    td_target = reward + discount_factor * np.dot(next_state, weights[:, next_action])
    td_error = td_target - np.dot(state, weights[:, action])
    weights[:, action] += learning_rate * td_error * state
    return weights


def train(env, num_episodes=10000, learning_rate=0.1, discount_factor=0.999, epsilon=0.3):
    num_actions = env.action_space.n
    num_features = env.observation_space.shape[0]  # Assuming Box space
    num_features = len(preprocess_state(DUMMY_STATE))  # Use an example or dummy initial state
    weights = np.zeros((num_features, num_actions))

    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        count=0
        while not done:
            processed_state = preprocess_state(state)  # Process state
            action = choose_action(processed_state, weights, epsilon, num_actions)
            next_state= env.step(action)
            total_reward += next_state[1]

            processed_next_state = preprocess_state(next_state)  # Process next state
            weights = update_weights(processed_state, action, next_state[1], processed_next_state, weights, learning_rate, discount_factor)

            state = next_state
            # found cookie break
            if next_state[1]>0:
                break
            # missed 5 cookies (MAX_TIME), minus points and break
            if count>MAX_TIME:
                # env._cumulative_reward+=PUNISHMENT
                # total_reward+=PUNISHMENT
                break
            count+=1
            # if count>100:
            #     print('count',count)
            #     break
            
            if done:
                print(f"DONE! Episode {episode}, count {count}, Total Reward: {total_reward}")
        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

    print("Training finished.")
    return weights

def test(env, weights, num_episodes=100):
    state = env.reset()

    total_rewards = []
    for _ in range(num_episodes):
        done = False
        total_reward = 0

        # while not done:
        processed_state = preprocess_state(state)  # Process state
        action = choose_action(processed_state, weights, 0, len(weights[0]))
        state = env.step(action)
        total_reward += state[1]
            # if state[1]>0:
            #     # print(f'Episoned {episode}, count {count}, reward {next_state[1]}')
            #     break

        total_rewards.append(total_reward)

    avg_reward = np.mean(total_rewards)
    print(f"Average reward over {num_episodes} episodes: {avg_reward}")

env = gym.make('cookiedisaster/GridWorld-v0')

# Train the agent
trained_weights = train(env)

Episode 0, Total Reward: -5.316062190712567
Episode 100, Total Reward: 0.5
Episode 200, Total Reward: 0.3327504983225351
Episode 300, Total Reward: 1
Episode 400, Total Reward: -0.9974756767772084
Episode 500, Total Reward: 1
Episode 600, Total Reward: -1.089508730677816
Episode 700, Total Reward: -1.0
Episode 800, Total Reward: 0.0
Episode 900, Total Reward: -1.0
Episode 1000, Total Reward: 0.5
Episode 1100, Total Reward: 1
Episode 1200, Total Reward: 0.5
Episode 1300, Total Reward: 0.5
Episode 1400, Total Reward: 1
Episode 1500, Total Reward: 1
Episode 1600, Total Reward: -2.5
Episode 1700, Total Reward: -2.5
Episode 1800, Total Reward: 0.5
Episode 1900, Total Reward: 1
Episode 2000, Total Reward: 1
Episode 2100, Total Reward: 1
Episode 2200, Total Reward: 0.5
Episode 2300, Total Reward: 1
Episode 2400, Total Reward: 1
Episode 2500, Total Reward: 1
Episode 2600, Total Reward: -2.0773491004469835
Episode 2700, Total Reward: 1
Episode 2800, Total Reward: 0.5
Episode 2900, Total Reward:

In [17]:

env = gym.make('cookiedisaster/GridWorld-v0',render_mode='human')

# Test the agent
test(env, trained_weights)
env.close()

Average reward over 100 episodes: -0.06888766166276471


In [8]:
trained_weights

array([[-0.15253852, -0.24189452, -0.23474692],
       [ 0.19079199, -0.3134597 , -0.70424752],
       [-0.16657725, -0.0258768 , -0.20355156],
       [ 0.43376501,  0.28852999,  0.19494037],
       [ 0.01403873, -0.21601772, -0.03119535],
       [ 0.41139576,  0.36041718, -0.20901164]])

# chat Neural network !

In [9]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, action_size)
    
    def forward(self, state):
        x = self.fc1(state)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [10]:
env = gym.make('cookiedisaster/GridWorld-v0')
state_size = len(preprocess_state(DUMMY_STATE))  # Adjust this based on your preprocessing
action_size = env.action_space.n
q_network = QNetwork(state_size, action_size)
optimizer = optim.Adam(q_network.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

In [11]:
def train_q_network(env, q_network, episodes=1000, gamma=0.999):
    for episode in range(episodes):
        state = env.reset()
        state = preprocess_state(state)  # Make sure to implement this
        done = False
        total_reward = 0
        count=0
        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            q_values = q_network(state_tensor)
            action = torch.argmax(q_values).item()
            
            next_state = env.step(action)
            done=next_state[2]
            reward = next_state[1]
            
            # found cookie break
            if next_state[1]>0:
                break
            # missed 5 cookies (MAX_TIME), minus points and break
            if count>MAX_TIME:
                # env._cumulative_reward+=
                break
            count+=1

            next_state = preprocess_state(next_state)
            next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)

            if not done:
                max_next_q = torch.max(q_network(next_state_tensor))
                target_q = reward + gamma * max_next_q
            else:
                target_q = torch.tensor([reward])

            target_q_values = q_values.clone()
            target_q_values[0][action] = target_q

            loss = loss_fn(q_values, target_q_values)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state = next_state
            total_reward += reward
        if episode % 100 == 0:
            print(f'Episode: {episode}, Total Reward: {total_reward}')
    return q_network


def test_q_network(env, q_network, episodes=100):
    state = env.reset()
    total_rewards = []
    for _ in range(episodes):
        
        done = False
        total_reward = 0

        
        state = preprocess_state(state)
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = q_network(state_tensor)
        action = torch.argmax(q_values).item()

        next_state= env.step(action)
        total_reward += next_state[1]
        state = next_state

        total_rewards.append(total_reward)

    avg_reward = np.mean(total_rewards)
    return avg_reward


In [12]:
env = gym.make('cookiedisaster/GridWorld-v0')
trained_network=train_q_network(env, q_network)

c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:131: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\spaces\box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:131: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'dict'>
  logger.warn(
c:\Users\saraa\anaconda3\envs\inf368\Lib\site-packages\gymnasium\utils\passive_env_checker.py:

Episode: 0, Total Reward: 0
Episode: 100, Total Reward: -18.933618826467114
Episode: 200, Total Reward: -2.5
Episode: 300, Total Reward: -2.5
Episode: 400, Total Reward: -17.51606219071256
Episode: 500, Total Reward: -18.933618826467114
Episode: 600, Total Reward: -2.5
Episode: 700, Total Reward: -2.5
Episode: 800, Total Reward: -2.5
Episode: 900, Total Reward: -2.5


In [18]:
env=gym.make('cookiedisaster/GridWorld-v0',render_mode='human')
test_q_network(env, trained_network)
env.close()